# Deep Learning for Time Series Analysis with PyTorch

This notebook demonstrates how to use PyTorch for time series forecasting using LSTM (Long Short-Term Memory) networks with NVIDIA CUDA GPU acceleration support.

## Topics Covered:
- NVIDIA CUDA/GPU detection and setup
- Generating dummy time series data
- Building an LSTM model for time series prediction
- Training the model
- Making predictions
- Visualizing results

## 1. Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 2. NVIDIA CUDA/GPU Setup

Check if NVIDIA CUDA is available and set the device accordingly for GPU acceleration.

In [ ]:
# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("CUDA not available, using CPU")

## 3. Generate Dummy Time Series Data

We'll create synthetic time series data with a trend, seasonality, and noise.

In [ ]:
def generate_time_series(n_samples=1000, seq_length=50):
    """
    Generate synthetic time series data with trend and seasonality.
    
    Args:
        n_samples: Total number of data points
        seq_length: Length of each sequence for training
    
    Returns:
        data: Generated time series array
    """
    time = np.arange(n_samples)
    
    # Trend component
    trend = 0.02 * time
    
    # Seasonal component (daily and weekly patterns)
    seasonal_daily = 10 * np.sin(2 * np.pi * time / 24)
    seasonal_weekly = 5 * np.sin(2 * np.pi * time / 168)
    
    # Noise
    noise = np.random.normal(0, 2, n_samples)
    
    # Combine all components
    data = trend + seasonal_daily + seasonal_weekly + noise + 50
    
    return data

# Generate data
data = generate_time_series(n_samples=2000)

# Visualize the generated time series
plt.figure(figsize=(15, 5))
plt.plot(data[:500])
plt.title('Generated Time Series Data (First 500 Points)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

print(f"Data shape: {data.shape}")
print(f"Data range: [{data.min():.2f}, {data.max():.2f}]")

## 4. Prepare Data for Training

Create sequences and normalize the data.

In [ ]:
class TimeSeriesDataset(Dataset):
    """
    PyTorch Dataset for time series data.
    """
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.data) - self.seq_length
    
    def __getitem__(self, idx):
        # Get sequence and next value
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + self.seq_length]
        return torch.FloatTensor(x), torch.FloatTensor([y])

# Normalize data
data_mean = data.mean()
data_std = data.std()
data_normalized = (data - data_mean) / data_std

# Split into train and test sets
train_size = int(len(data_normalized) * 0.8)
train_data = data_normalized[:train_size]
test_data = data_normalized[train_size:]

# Create datasets
seq_length = 50
train_dataset = TimeSeriesDataset(train_data, seq_length)
test_dataset = TimeSeriesDataset(test_data, seq_length)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")
print(f"Batch size: {batch_size}")
print(f"Sequence length: {seq_length}")

## 5. Define LSTM Model

Build a neural network with LSTM layers for time series prediction.

In [ ]:
class LSTMModel(nn.Module):
    """
    LSTM-based model for time series forecasting.
    """
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Reshape input to [batch_size, seq_length, input_size]
        x = x.unsqueeze(-1)
        
        # LSTM forward pass
        # lstm_out shape: [batch_size, seq_length, hidden_size]
        lstm_out, (hidden, cell) = self.lstm(x)
        
        # Use the last output for prediction
        # Shape: [batch_size, hidden_size]
        last_output = lstm_out[:, -1, :]
        
        # Fully connected layer
        prediction = self.fc(last_output)
        
        return prediction

# Create model instance
model = LSTMModel(input_size=1, hidden_size=64, num_layers=2, output_size=1).to(device)

# Print model architecture
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

## 6. Training Setup

Define loss function, optimizer, and training parameters.

In [ ]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Training parameters
num_epochs = 50

print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer}")
print(f"Number of epochs: {num_epochs}")

## 7. Train the Model

Train the LSTM model on the time series data.

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs, device):
    """
    Train the LSTM model.
    """
    train_losses = []
    test_losses = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_x, batch_y in train_loader:
            # Move data to device
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation phase
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                test_loss += loss.item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        
        # Update learning rate
        scheduler.step(test_loss)
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
    
    return train_losses, test_losses

# Train the model
print("Starting training...\n")
train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs, device)
print("\nTraining completed!")

## 8. Visualize Training Progress

In [ ]:
# Plot training and test losses
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Test Loss Over Time')
plt.legend()
plt.grid(True)
plt.show()

print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Test Loss: {test_losses[-1]:.4f}")

## 9. Make Predictions

In [ ]:
def predict_future(model, initial_sequence, n_steps, device):
    """
    Predict future values using the trained model.
    
    Args:
        model: Trained LSTM model
        initial_sequence: Initial sequence to start predictions
        n_steps: Number of future steps to predict
        device: Device to run predictions on
    
    Returns:
        predictions: Array of predicted values
    """
    model.eval()
    predictions = []
    
    current_sequence = torch.FloatTensor(initial_sequence).unsqueeze(0).to(device)
    
    with torch.no_grad():
        for _ in range(n_steps):
            # Predict next value
            pred = model(current_sequence)
            predictions.append(pred.cpu().item())
            
            # Update sequence with prediction
            current_sequence = torch.cat([current_sequence[:, 1:], pred.unsqueeze(1)], dim=1)
    
    return np.array(predictions)

# Get initial sequence from test data
initial_seq = test_data[:seq_length]
n_future_steps = 100

# Make predictions
predictions = predict_future(model, initial_seq, n_future_steps, device)

# Denormalize predictions and actual data
predictions_denorm = predictions * data_std + data_mean
actual_denorm = test_data[seq_length:seq_length + n_future_steps] * data_std + data_mean

print(f"Generated {n_future_steps} future predictions")

## 10. Visualize Predictions vs Actual Values

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(15, 6))
plt.plot(range(len(actual_denorm)), actual_denorm, label='Actual', linewidth=2)
plt.plot(range(len(predictions_denorm)), predictions_denorm, label='Predicted', linewidth=2, alpha=0.7)
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.title('Time Series Prediction: Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()

# Calculate prediction accuracy metrics
mse = np.mean((actual_denorm - predictions_denorm) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(actual_denorm - predictions_denorm))

print(f"\nPrediction Metrics:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

## 11. Model Evaluation Summary

In [ ]:
# Evaluate model on entire test set
model.eval()
all_predictions = []
all_actuals = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        all_predictions.extend(outputs.cpu().numpy())
        all_actuals.extend(batch_y.numpy())

all_predictions = np.array(all_predictions).flatten()
all_actuals = np.array(all_actuals).flatten()

# Denormalize
all_predictions_denorm = all_predictions * data_std + data_mean
all_actuals_denorm = all_actuals * data_std + data_mean

# Calculate metrics
final_mse = np.mean((all_actuals_denorm - all_predictions_denorm) ** 2)
final_rmse = np.sqrt(final_mse)
final_mae = np.mean(np.abs(all_actuals_denorm - all_predictions_denorm))

print("="*50)
print("MODEL EVALUATION SUMMARY")
print("="*50)
print(f"Device Used: {device}")
print(f"Model Architecture: LSTM with {model.num_layers} layers")
print(f"Hidden Size: {model.hidden_size}")
print(f"Training Samples: {len(train_dataset)}")
print(f"Testing Samples: {len(test_dataset)}")
print(f"\nFinal Metrics on Test Set:")
print(f"  MSE:  {final_mse:.4f}")
print(f"  RMSE: {final_rmse:.4f}")
print(f"  MAE:  {final_mae:.4f}")
print("="*50)

## 12. Save the Trained Model (Optional)

In [ ]:
# Save model checkpoint
model_path = 'lstm_time_series_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'test_losses': test_losses,
    'data_mean': data_mean,
    'data_std': data_std,
    'seq_length': seq_length
}, model_path)

print(f"Model saved to {model_path}")

# To load the model later:
# checkpoint = torch.load(model_path)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

## Summary

This notebook demonstrated:
1. ✅ CUDA/GPU detection and configuration for PyTorch
2. ✅ Generation of synthetic time series data with trend and seasonality
3. ✅ Building an LSTM neural network for time series forecasting
4. ✅ Training the model with proper data loading and batching
5. ✅ Making predictions on future time steps
6. ✅ Visualizing results and evaluating model performance
7. ✅ Saving and loading model checkpoints

### Next Steps:
- Experiment with different model architectures (GRU, Transformer)
- Try different hyperparameters (hidden size, number of layers, learning rate)
- Use real-world time series data
- Implement multi-step ahead forecasting
- Add attention mechanisms for improved performance